In [1]:
from neo4j import GraphDatabase
import pandas as pd
import csv
import json

uri = "neo4j+s://b12269b4.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j","gQKq-ohMlV4pVgXMLtkpPYkryQ2q7-zRlaP5fNKX744"))


#Had to check the connection, took this from chatgtp
def ping_neo4j():
    try:
        driver = GraphDatabase.driver(uri, auth=("neo4j", "gQKq-ohMlV4pVgXMLtkpPYkryQ2q7-zRlaP5fNKX744"))
        with driver.session() as session:
            # En enkel Cypher-fråga för att testa anslutningen
            result = session.run("RETURN 1 AS ping")
            for record in result:
                if record["ping"] == 1:
                    print("Anslutning till Neo4j fungerar!")
                    return True
        print("Anslutning misslyckades!")
        return False
    except Exception as e:
        print(f"Fel vid anslutning: {e}")
        return False
    finally:
        driver.close()

ping_neo4j()

Anslutning till Neo4j fungerar!


True

In [2]:
with open('C:/Users/theod/OneDrive/Skrivbord/Tuc/VSCode/Databastyper/Kunskapkontroll_1/Neo4j/prepared/movies.json', encoding='utf-8') as inputfile:
    df_movies = pd.read_json(inputfile)

df_movies.to_csv('C:/Users/theod/OneDrive/Skrivbord/Tuc/VSCode/Databastyper/Kunskapkontroll_1/Neo4j/prepared/movies.csv', encoding='utf-8', index=False)

In [9]:
new_df = df_movies.convert_dtypes()

columns_to_remove = [
    'num_critic_for_reviews', 'gross', 'num_voted_users', 'facenumber_in_poster', 'plot_keywords',
    'budget', 'title_year', 'aspect_ratio','movie_imdb_link','num_user_for_reviews']

columns_to_remove = [col for col in columns_to_remove if col in new_df.columns]
if columns_to_remove:
    new_df = new_df.drop(columns=columns_to_remove)
# new_df

c:\Users\theod\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\theod\anaconda3\Lib\site-packages\pandas\core\dtypes\cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [10]:
actor_df = new_df.explode('actors')
actor_df[['actors', 'movie_id', 'movie_title']].to_csv('actors_movies.csv', index=False)
# actor_df.head()

MATCH (a:Actors)
MATCH (m:Movies)
WHERE a.movie_title = m.movie_title
MERGE (a)-[:ACTED_IN]->(m);

In [11]:
genre_df = new_df.explode('genres')
genre_df[['genres', 'movie_id','movie_title']].to_csv('genres_movies.csv', index=False)
# genre_df.head()

MATCH (g:Genre)
MATCH (m:Movies)
WHERE g.movie_title = m.movie_title
MERGE (g)-[:HAS_GENRE]->(m);

MATCH (g:Genre)-[:HAS_GENRE]->(m:Movies)
WITH m, collect(g.genres) AS genre_combination
RETURN genre_combination, count(*) AS count
ORDER BY count DESC LIMIT 10;

MATCH (g:Genre)-[:HAS_GENRE]->(m:Movies)
WITH m, collect(g.genres) AS genre_combination
WHERE m.movie_title = m.movie_title
RETURN genre_combination, count(*) AS count
ORDER BY count DESC;

In [13]:
directors_df = new_df.explode('director_name')
genre_df[['director_name', 'movie_id','movie_title', 'duration']].to_csv('director_name_movies.csv', index=False)
#directors_df.head()

DIRECTED
MATCH (d:Director)
MATCH (m:Movies)
WHERE d.movie_id = m.movie_id
MERGE (d)-[:DIRECTED]->(m);

In [14]:
movie_titles_df = new_df.explode('movie_title')
genre_df[['movie_id','movie_title', 'director_name','duration']].to_csv('movie_title_movies.csv', index=False)
# movie_titles_df.head()

när du kommer till the lord of rings samma probles som tidigare uppgift med blanksteg i titlen.

MATCH (m:Movies) WHERE m.movie_title =~ "The Lord of the Rings: The Fellowship of the Ring." OR m.movie_title =~ "The Lord of the Rings: The Two Towers." OR m.movie_title =~ "The Lord of the Rings: The Return of the King.*" MERGE (a:Actors {actors: "Viggo Mortensen"}) MERGE (a)-[:ACTED_IN]->(m);